In [1]:
import random
from bisect import bisect_left

LASTTIME = 1000000;
LAMDAS = [0.01, 0.1, 0.3, 0.5] #the fault generation rates at each layer
I = [1, 2, 3, 4, 5] #layer i starting from 1
N = [0, 100, 100, 100, 100, 100] #number of elements for layer i, 0 for ignored layer 0
M = [0, 30, 30, 30, 30, 30] #select number of elements to monitor out of N for layer i, 0 for ignored layer 0
SEED = 1
random.seed(SEED)

#Counter for monitored/detected/diagnosed faults: 
C_M = 0
C_D = 0
C_R = 0


#Generate N elements for each layer and select M out of N
class Element: #define a element
 def __init__(self, eID, layer, monitored):
    self.id = eID
    self.layer = layer
    self.monitored = monitored

 def infoOut(self):
    print("Element info: id = "
          + str(self.id)
          + ", layer = " 
          + str(self.layer) 
          + ", monitored = " 
          + str(self.monitored) 
         ) 

elements = []
elements.append([]) #empty for ignored layer 0
elementID = 0
for i in I: #for each layer i
    j = 0
    elements_layer = []
    while j < N[i]: #generate Ni elements
        elementID = elementID + 1
        elements_layer.append(Element(elementID, i, False))
        j = j + 1
    count = 0
    assert(M[i] <= N[i]), "Total elements Ni is too small to reach Mi!"
    while count < M[i]: #select Mi elements out of Ni 
        randomElement = random.randrange(N[i])
        if elements_layer[randomElement].monitored == False:
            elements_layer[randomElement].monitored = True
            count = count + 1
    elements.append(elements_layer)
    
    #print(M[i])
    #print(N[i])
    #for e in element_layer:
    #    e.infoOut()  
#print(len(elements))

#Generate a seqence of faults sorted in an increasing order of evtTime
class Fault: #define a fault
 def __init__(self, fID, layer, initTime, evtTime, assocElement, impactVector):
    self.id = fID
    self.layer = layer
    self.initTime = initTime #the inital time when the fault is generated for calculating delay
    self.evtTime = evtTime #the time when the fault is re-inserted in the event queue
    self.assocElement = assocElement #within N elements
    self.impactVector = impactVector #impact vector of this fault
    
 def infoOut(self):
    print("Fault info: id = "
          + str(self.id)
          + ", layer = " 
          + str(self.layer) 
          + ", initTime = " 
          + str(self.initTime) 
          + ", evtTime = " 
          + str(self.evtTime) 
          + ", assocElementID = " 
          + str(self.assocElement.id)
          + ", monitored = " 
          + str(self.assocElement.monitored)
          + ", impactVector = " 
          + str(self.impactVector)          
         )
    
def insert(seq, keys, item, keyfunc=lambda v: v):
    """Insert an item into a sorted list using a separate corresponding
       sorted keys list and a keyfunc() to extract the key from each item.

    Based on insert() method in SortedCollection recipe:
    http://code.activestate.com/recipes/577197-sortedcollection/
    """
    k = keyfunc(item)  # Get key.
    i = bisect_left(keys, k)  # Determine where to insert item.
    keys.insert(i, k)  # Insert key of item to keys list.
    seq.insert(i, item)  # Insert the item itself in the corresponding place.
    

evtID = 0
queue = [] #a queue of faults 

# for each layer, insert the initial fault with evtTime = 0 in the queue
for i in I: 
    evtTime = 0
    evtID = evtID + 1
    elements_layer = elements[i] #find corresponding elements_layer 
    
    elementIndex = random.randrange(len(elements_layer))#randomly (uniformaly) assign fault to Ni elements at layer i
    newFault = Fault(evtID, i, evtTime, evtTime, 
                     elements_layer[elementIndex], random.random()) #randomly generate a double number between 0 and 1 for impact vector
    keys = [fault.evtTime for fault in queue]
    insert(queue, keys, newFault, keyfunc=lambda x: x.evtTime) #insert on the order of evtTime to the queue
    #newFault.infoOut()
#print(len(queue)) 
#for f in queue:
#    f.infoOut()   
    
#start to process a Head of Line fault in the event queue
while (len(queue) > 0):
    currFault = queue[0]
    #currFault.infoOut()
    queue.remove(currFault) #delete the currFault 

    #insert the next new fault based on the info of the currFault
    evtTime = currFault.evtTime + random.expovariate(LAMDAS[1])
    if evtTime <= LASTTIME:
        evtID = evtID + 1
        elementIndex = random.randrange(len(elements[currFault.layer])) #generate a new fault with the same layer as the current fault
        newFault = Fault(evtID, currFault.layer, evtTime, evtTime, 
                         elements[currFault.layer][elementIndex], random.random())     
        keys = [fault.evtTime for fault in queue]
        insert(queue, keys, newFault, keyfunc=lambda x: x.evtTime)
        #newFault.infoOut()
    
    #add code here to process the currFault
    #Calculate counters
    if currFault.assocElement.monitored == True:
        C_M = C_M + 1
        
print(str(C_M))
print(str(evtID))
print("The ratio of monitored faults = " + str(C_M/evtID))
    


150577
501467
The ratio of monitored elements = 0.30027299902087273
